In [1]:
import numpy as np

In [3]:
import pyphi

In [5]:
from itertools import chain,combinations

In [6]:
from scipy import stats

![title](phi_algorithm.png)

In [97]:
class phi():
    def __init__(self,tpm,states=None):
        self.tpm = tpm
        self.num_nodes = int(np.log2(len(tpm)))
        if states is None:
            self.states = np.zeros((self.num_nodes))
        else:
            self.states = states
        
    # Converts a list of states to a row number, so that we can index the tpm
    # input : list of states, Ex. [0,1,0,0]
    # output : row number based on little endian notation, so index zero in list is least
    #          significant bit and index n-1 is most significant bit. Ex. [0,1,0,0] -> 2
    def state_to_index(self,states):
        decimal = 0
        for i,state in enumerate(states):
            decimal += (2**i) * (state)
        return decimal
    
    # Converts a row number from the tpm into a list of states
    # input : row number, total number of states. Ex. (2, 4)
    # output : list of states that row number represents. Ex. (2, 4) -> [0,1,0,0]
    #          We include the number of states so that we know how many zeros to append at the end.
    def index_to_state(self,index,num_states):
        binary = bin(index)[2:]
        states = []
        for state in binary: # convert row to binary
            states.append(int(state))
        
        states = np.flip(states) # flip to make little endian
       
        if (len(states) < num_states): 
            states = np.concatenate((states,np.zeros((num_states-len(states)),dtype=int)))
            
        return states
    
    # Because they're conditional probability distributions, the rows need to sum to one
    def normalize_rows(self,tpm):
        return tpm/np.sum(tpm,1)[:, np.newaxis]
    
    # return the powerset of a list of nodes
    def powerset(self,iterable):
        "powerset([1,2,3]) --> (1,) (2,) (3,) (1,2) (1,3) (2,3)"
        s = list(iterable)
        return chain.from_iterable(combinations(s, r) for r in range(1,len(s)))
    
    # return the powerset of a list of nodes including full set
    def powerset_all(self,iterable):
        "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
        s = list(iterable)
        return chain.from_iterable(combinations(s, r) for r in range(0,len(s)+1))
    
    # returns the tensor product of two tpm (effect)
    def tensor_product(self,t1,t2):
        tensor_product = np.zeros((t1.shape[0],t1.shape[1]*t2.shape[1]))
        column = 0
        for c2 in t2.T:
            for c1 in t1.T:
                tensor_product[:,column] = c1*c2
                column+=1
                
        return tensor_product
    
    # This tensor product combines two effect repertoires t1 and t2, given a list of their nodes also
    # This ensures that the nodes get ordered correctly even if the purview nodes of t1 are 'B' and the
    # purview nodes of t2 are 'A,C'.
    def tensor_product_ordered(self,t1,t1_nodes,t2,t2_nodes):
        tensor_product = np.zeros((t1.shape[0],t1.shape[1]*t2.shape[1]))

        columns = tensor_product.shape[1]
        
        # Fill the columns of the tpm one at a time
        for column in range(columns):
            # find the states of t1's nodes given the column we're currently filling
            t1_state = self.index_to_state(column,int(np.log2(columns)))[t1_nodes]
            # do the same for t2's nodes
            t2_state = self.index_to_state(column,int(np.log2(columns)))[t2_nodes]
            # Then fill the column with the correct column from t1 multiplied by the correct column from
            # t2
            tensor_product[:,column] = t1[:,self.state_to_index(t1_state)] * t2[:,self.state_to_index(t2_state)]
            
        return tensor_product
        
    """
    effect_repertoire :
        input : mechanism (t nodes), purview (t+1 nodes)
        algorithm : from original tpm find the tpm P(purview | mechanism)
        output : repertoire for those nodes
    """    
    def effect_repertoire(self,mechanism,purview):
        # The effect repertoire captures the conditional transition probability of transitioning to each
        # purview state, given the current mechanism state, so it needs to be of size
        # NUMBER POSSIBLE MECHANISM STATES X NUMBER POSSIBLE PURVIEW STATES, but we start by marginalizing
        # over only the mechanism states, so NUMBER POSSIBLE MECHANISM STATES X NUMBER POSSIBLE STATES
        mechanism_effect_repertoire = np.zeros((2**len(mechanism),2**self.num_nodes))
        
        # We marginalize over the mechanism states, which means we sum the probabilities of rows which
        # only differ in the state of nodes not in the mechanism.
        # We do this by finding the mechanism's state for a given row and mapping that row in the original tpm
        # to the correct row in the new mechanism repertoire
        for row in range(self.tpm.shape[0]):
            mechanism_state = self.index_to_state(row,self.num_nodes)[mechanism]
            mechanism_effect_repertoire[self.state_to_index(mechanism_state),:] += self.tpm[row,:]
        
        # This is the final effect repertoire
        effect_repertoire = np.zeros((2**len(mechanism),2**len(purview)))
        
        # Second, we marginalize over the column states, which means we sum the probabilities of columns which
        # only differ in the state of nodes not in the purview.
        # We do this by finding the purview's state for a given column and mapping that row in the original tpm
        # to the correct column in the new effect repertoire
        for column in range(self.tpm.shape[1]):
            purview_state = self.index_to_state(column,self.num_nodes)[purview]
            effect_repertoire[:,self.state_to_index(purview_state)] += mechanism_effect_repertoire[:,column]
        
        # All that's left to do is normalize the rows because each row is a conditional probability distribution
        effect_repertoire = self.normalize_rows(effect_repertoire)
        
        # Now, we have to expand the effect_repertoire into the original state space which has all the 
        # possible current states at time t
        expanded_effect_repertoire = np.zeros((2**self.num_nodes,2**len(purview)))
        
        # This is done by mapping distributions in the effect_repertoire to each row in the expanded repertoire
        # where the mechanism's state matches
        for row in range(2**self.num_nodes):
            mechanism_state = self.index_to_state(row,self.num_nodes)[mechanism]
            expanded_effect_repertoire[row,:] = effect_repertoire[self.state_to_index(mechanism_state),:]
            
        return expanded_effect_repertoire
    
    def effect_mip(self,system,purview):        
        # We need to find the cut the makes the least difference to the tpm of the system,
        # so we generate the original tpm first. We'll want to compare the original tpm
        # with the tpm generated after making our cut to determine the cut that makes the
        # least difference.
        min_cut = None
        cut_distance = float('inf')
        full_tpm = self.effect_repertoire(system,purview)
        
        # We need to loop through two powersets to determine all the cuts for this particular 
        # system + purview pair. We need to loop through the system powerset and the purview powerset
        # See factors.png below for an example.
        
        seen = set()
        
        for system_subset in self.powerset_all(system):
            
            for purview_subset in self.powerset_all(purview):
                
                # We haven't already seen the complement of the subset and the length of both of the factors
                # aren't zero
                if system_subset not in seen and not (len(system_subset) == 0 and len(purview_subset) == 0):
                    system_factor_1 = list(system_subset)
                    purview_factor_1 = list(purview_subset)
                    
                    system_factor_2 = list(np.setdiff1d(system,system_subset))
                    purview_factor_2 = list(np.setdiff1d(purview,purview_subset))
                    
                    tpm_1 = self.effect_repertoire(system_factor_1,purview_factor_1)
                    tpm_2 = self.effect_repertoire(system_factor_2,purview_factor_2)
                    
                    cut_tpm = self.tensor_product_ordered(tpm_1,purview_factor_1,tpm_2,purview_factor_2)
                    
                    # assess the distance between the cut_tpm and the full_tpm
                    new_cut_distance = stats.wasserstein_distance(full_tpm[self.state_to_index(self.states)],
                                                         cut_tpm[self.state_to_index(self.states)])
                    
                    print ((system_factor_1,purview_factor_1,system_factor_2,purview_factor_2)," ",new_cut_distance)
                    
                    # update the distance with the lowest distance
                    if new_cut_distance <= cut_distance:
                        cut_distance = new_cut_distance
                        min_cut = (system_factor_1,purview_factor_1,system_factor_2,purview_factor_2)
            
            # add the complement of the system_subset to the seen set to ensure we don't double count certain
            # partitions
            seen.add(tuple(np.setdiff1d(system,system_subset)))
        
        return cut_distance,min_cut
    
    # TODO: implement this. Right now it's not working when the purview is smaller than the full
    # list of nodes because the tensor product assumes the second dimension will be the full length
    # of all the states given all the nodes
    def mie(self,system):
        phi = float('-inf')
        mie = None
        for purview in self.powerset_all(np.arange(self.num_nodes)):
            print (list(purview))
            cut_distance,min_cut = self.effect_mip(system,list(purview))
            if cut_distance > phi:
                phi = cut_distance
                mie = min_cut
        return phi,mie
    
    """        
    mie : 
        input : tpm for a specific system (in our case this is the entire system)
        
        algorithm : for each subset of nodes at t+1 of the system (call this a purview):
                        call effect_mip(system, purview)
                    return max effect_mip
        
        output : effect phi value
                        
        
    concept :
        input : list of nodes in system (in our case this will be the whole system)
        
        algorithm : min (mic(system),mie(system))
        
        output : phi value for that system
    """

![title](factors.png)

In [98]:
test = phi(np.array([[1., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 1., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 1., 0., 0.],
                   [0., 1., 0., 0., 0., 0., 0., 0.],
                   [0., 1., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 0., 0., 1.],
                   [0., 0., 0., 0., 0., 1., 0., 0.],
                   [0., 0., 0., 1., 0., 0., 0., 0.]], dtype=int),[1,0,0])

In [99]:
test.tpm

array([[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0]])

In [100]:
test.effect_mip([0,1,2],[0,1,2])

([], [0], [0, 1, 2], [1, 2])   0.0625
([], [1], [0, 1, 2], [0, 2])   0.0625
([], [2], [0, 1, 2], [0, 1])   0.125
([], [0, 1], [0, 1, 2], [2])   0.125
([], [0, 2], [0, 1, 2], [1])   0.15625
([], [1, 2], [0, 1, 2], [0])   0.15625
([], [0, 1, 2], [0, 1, 2], [])   0.1875
([0], [], [1, 2], [0, 1, 2])   0.125
([0], [0], [1, 2], [1, 2])   0.15625
([0], [1], [1, 2], [0, 2])   0.1875
([0], [2], [1, 2], [0, 1])   0.125
([0], [0, 1], [1, 2], [2])   0.1875
([0], [0, 2], [1, 2], [1])   0.125
([0], [1, 2], [1, 2], [0])   0.1875
([0], [0, 1, 2], [1, 2], [])   0.1875
([1], [], [0, 2], [0, 1, 2])   0.125
([1], [0], [0, 2], [1, 2])   0.1875
([1], [1], [0, 2], [0, 2])   0.15625
([1], [2], [0, 2], [0, 1])   0.1875
([1], [0, 1], [0, 2], [2])   0.1875
([1], [0, 2], [0, 2], [1])   0.1875
([1], [1, 2], [0, 2], [0])   0.1875
([1], [0, 1, 2], [0, 2], [])   0.1875
([2], [], [0, 1], [0, 1, 2])   0.125
([2], [0], [0, 1], [1, 2])   0.1875
([2], [1], [0, 1], [0, 2])   0.125
([2], [2], [0, 1], [0, 1])   0.1875
([2], 

(0.0625, ([], [1], [0, 1, 2], [0, 2]))